# Стратегия входа в зоне ценности и выхода при смене тренда

* Здесь мы протестируем стратегию покупки акций в зоне ценности. 
* Данная зона находится между двумя скользящими средними - быстрой и медленной.
* Преимущество зоны ценности заключается в том, что она позволяет не переплачивать за актив. 
* Подробнее об условиях стратегии и результах ее бэктестинга читайте здесь: https://mindspace.ru/36256-testiruem-strategiyu-vhoda-v-zone-tsennosti-s-kodom-na-python/

In [ ]:
# импортирует библиотеку talib для построения скользящей средней
# импортирует библиотеку numpy для расчета позиции

import talib
import numpy as np

def initialize(context):
   """
    Создаем функцию initialize(context) для начала работы.
    Указываем нужный тикер актива через свойство context.security.
    Создаем функцию schedule_function(), в которой проверяем позицию каждый день через 1 час после открытия рынка.
    
    """
    context.security = symbol('SPY')
    schedule_function(rebalance, date_rules.every_day(), time_rules.market_open(hours=1))

    # готовимся собирать статистику по позициям (числе сделок, времени удержания)
    init_test_stat(context)
    
def rebalance(context, data):
    """
    Создаем функцию rebalance(context, data).
    Здесь проверяются сигналы и собирается статистика нахождения в позиции.
    """
    test_stat(context, 'total_days')
    
    # получаем истории цен закрытия актива за последние 500 дней
    
    hist = data.history(context.security, ['high', 'low', 'close'], 500, '1d')
    
    # проверяем наличие бычьего тренда в акции для входа на лонг
    # строим на основе talib простые скользящие SMA по ценам закрытия за 200 и 50 дней для определения тренда
    # задаем переменные для расчета средних для зоны ценности - быстрой и медленной
    
    price = hist['close'][-1]
    ma_200 = talib.SMA(hist['close'], timeperiod=200)
    ma_50 = talib.SMA(hist['close'], timeperiod=50)
    ma_low = talib.EMA(hist['close'], timeperiod=26)
    ma_fast = talib.EMA(hist['close'], timeperiod=13)
        
        
    if  ma_50[-1] < ma_200[-1]:
        order_target_percent(context.security, 0.)
        
    elif ma_50[-1] > ma_200[-1] and ma_low[-1] < price < ma_fast[-1]:
        order_target_percent(context.security, 1.)
        
    
    open_total = np.sum([x.amount * x.cost_basis for x in context.portfolio.positions.values()])
    record(open_total=open_total)
    
    # считаем дни с открытой позицией
    if open_total != 0:
        test_stat(context, 'total_days_in_position')
    
    # выводим статистику в конце
    print_stat(context)
    
def print_stat(context):
    # выводим лог в последний день
    if (get_datetime().date() == context.end_date):  
        print("stat: ", 
              context.stat, 
              float(context.stat['total_days_in_position']) / context.stat['total_days'])

        
    # накапливаем статистику
def test_stat(context, k):
    context.stat[k] += 1
    

    # готовимся собирать данные о времени в позиции и числе сделок
def init_test_stat(context):
    # берем дату окончания теста, чтобы вывести накопленные данные
    env = get_environment('*') 
    context.end_date = env['end'].date()
    
    # готовим массив для логов
    context.stat = {
        'total_days': 0,
        'total_days_in_position': 0,
        'trades_buy_executed': 0,
        'trades_sell_executed': 0,
    }